In [66]:
!pip install pyspark
!apt update
!apt install openjdk-8-jdk-headless -qq
import os
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext
from pyspark.sql import SQLContext
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:2 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:3 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Get:6 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Ign:7 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:8 https://r2u.stat.illinois.edu/ubuntu jammy Release
Hit:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Fetched 257 kB in 2s (160 kB/s)
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
54 packages can be upgraded. Run 'apt list --upgradable' to see them.
W: Skipping ac

In [2]:
spark = SparkSession.builder.getOrCreate()
sc = spark.sparkContext
sqlContext = SQLContext(sc)
df = sqlContext.read.csv('transactions_train.csv', header=True, inferSchema=True)
transactions_rdd = df.rdd

/usr/local/lib/python3.10/dist-packages/pyspark/sql/context.py:113: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


In [3]:
transactions_rdd.first()

Row(_c0=0, Id=2987000, target=0, timestamp=86400, ammount=68.5, product_category='W', feat_card_1=13926, feat_card_2=None, feat_card_3=150.0, feat_card_4='discover', feat_card_5=142.0, feat_card_6='credit', addr1=315.0, addr2=87.0, dist1=19.0, dist2=None, payer_email_domain=None, receiver_email_domain=None, A1=1.0, A2=1.0, A3=0.0, A4=0.0, A5=0.0, A6=1.0, A7=0.0, A8=0.0, A9=1.0, A10=0.0, A11=2.0, A12=0.0, A13=1.0, A14=1.0, B1=14.0, B2=None, B3=13.0, B4=None, B5=None, B6=None, B7=None, B8=None, B9=None, B10=13.0, B11=13.0, B12=None, B13=None, B14=None, B15=0.0, C1='T', C2='T', C3='T', C4='M2', C5='F', C6='T', C7=None, C8=None, C9=None, D1=1.0, D2=1.0, D3=1.0, D4=1.0, D5=1.0, D6=1.0, D7=1.0, D8=1.0, D9=1.0, D10=0.0, D11=0.0, D12=1.0, D13=1.0, D14=1.0, D15=0.0, D16=0.0, D17=0.0, D18=0.0, D19=1.0, D20=1.0, D21=0.0, D22=0.0, D23=1.0, D24=1.0, D25=1.0, D26=1.0, D27=0.0, D28=0.0, D29=0.0, D30=0.0, D31=0.0, D32=0.0, D33=0.0, D34=0.0, D35=None, D36=None, D37=None, D38=None, D39=None, D40=None, D

Porcentaje de fraude

In [ ]:
transactions_rdd.map(lambda x: x.target).distinct().collect()

[0, 1]

In [ ]:
transactions_rdd.filter(lambda x: x.target == 1).count() * 100 / transactions_rdd.count()

3.502278074504093

Porcentaje de monto fraudulento respecto de los productos

In [ ]:
fraude = transactions_rdd.filter(lambda x: x.target == 1).map(lambda x: (x.product_category, x.ammount)).reduceByKey(lambda a,b: a + b)

In [ ]:
legal = transactions_rdd.filter(lambda x: x.target == 0).map(lambda x: (x.product_category, x.ammount)).reduceByKey(lambda a,b: a + b)

In [ ]:
producto_monto_f_l = fraude.fullOuterJoin(legal).collect()

In [ ]:
def porcentaje(lista):
  promedios = []
  for producto in lista:
    promedios.append((producto[0], producto[1][0] * 100 / (producto[1][0] + producto[1][1])))
  return promedios

porcentaje(producto_monto_f_l)

[('H', 10.087869182508362),
 ('R', 5.134079848889628),
 ('S', 7.983558441966235),
 ('C', 12.916996422843251),
 ('W', 3.0548043859574134)]

Fraude segun el tiempo, analizando cantidad y monto

In [4]:
fraude = transactions_rdd.filter(lambda x: x.target == 1).cache()

In [60]:
def segundos(numero):
  return numero - 86400

def analisis(numero, divisor):
  flotante = (numero - 86400) / divisor
  entero = (numero - 86400) // divisor
  if flotante > entero:
    return entero + 1
  else:
    return entero

def minutos(numero):
  return analisis(numero, 60)

def horas(numero):
  return analisis(numero, 3600)

def dias(numero):
  return analisis(numero, 86400)

def semanas(numero):
  return analisis(numero, 604800)

def meses(numero):
  return analisis(numero, 2419200)

def porcentaje_tupla(tupla, total):
  return (tupla[0], tupla[1], tupla[1] * 100 / total)

def porcentaje_lista(lista, total):
  nueva_lista = []
  for elemento in lista:
    nueva_lista.append((elemento[0], elemento[1], elemento[1] * 100 / total))
  return nueva_lista

Cantidad respecto del tiempo

Cantidad total respecto de segundos

In [78]:
cantidad_total = fraude.count()
(fraude.map(lambda x: segundos(x.timestamp)).reduce(lambda a,b: a if a > b else b), cantidad_total, 100)

(13064705, 17580, 100)

Máxima cantidad respecto de minutos

In [79]:
porcentaje_tupla(fraude.map(lambda x: (minutos(x.timestamp), 1)).reduceByKey(lambda a,b: a + b).reduce(lambda a,b: a if a[1] > b[1] else b), cantidad_total)

(136426, 6, 0.034129692832764506)

Máxima cantidad respecto de horas

In [80]:
porcentaje_tupla(fraude.map(lambda x: (horas(x.timestamp), 1)).reduceByKey(lambda a,b: a + b).reduce(lambda a,b: a if a[1] > b[1] else b), cantidad_total)

(1529, 42, 0.23890784982935154)

Cantidad respecto de días

In [81]:
porcentaje_lista(sorted(fraude.map(lambda x: (dias(x.timestamp), 1)).reduceByKey(lambda a,b: a + b).collect()), cantidad_total)

[(1, 112, 0.6370875995449374),
 (2, 123, 0.6996587030716723),
 (3, 92, 0.5233219567690558),
 (4, 115, 0.6541524459613197),
 (5, 127, 0.7224118316268487),
 (6, 99, 0.5631399317406144),
 (7, 136, 0.7736063708759955),
 (8, 93, 0.5290102389078498),
 (9, 119, 0.676905574516496),
 (10, 90, 0.5119453924914675),
 (11, 123, 0.6996587030716723),
 (12, 84, 0.4778156996587031),
 (13, 82, 0.4664391353811149),
 (14, 126, 0.7167235494880546),
 (15, 143, 0.8134243458475541),
 (16, 70, 0.3981797497155859),
 (17, 126, 0.7167235494880546),
 (18, 161, 0.9158134243458476),
 (19, 118, 0.6712172923777019),
 (20, 135, 0.7679180887372014),
 (21, 116, 0.6598407281001137),
 (22, 147, 0.8361774744027304),
 (23, 106, 0.602957906712173),
 (24, 76, 0.4323094425483504),
 (25, 63, 0.3583617747440273),
 (26, 82, 0.4664391353811149),
 (27, 100, 0.5688282138794084),
 (28, 150, 0.8532423208191127),
 (29, 129, 0.7337883959044369),
 (30, 158, 0.8987485779294653),
 (31, 149, 0.8475540386803185),
 (32, 51, 0.2901023890784983)

Máxima cantidad respecto de días

In [82]:
porcentaje_tupla(fraude.map(lambda x: (dias(x.timestamp), 1)).reduceByKey(lambda a,b: a + b).reduce(lambda a,b: a if a[1] > b[1] else b), cantidad_total)

(64, 234, 1.3310580204778157)

Cantidad respecto de semanas

In [83]:
porcentaje_lista(sorted(fraude.map(lambda x: (semanas(x.timestamp), 1)).reduceByKey(lambda a,b: a + b).collect()), cantidad_total)

[(1, 804, 4.573378839590443),
 (2, 717, 4.078498293515358),
 (3, 869, 4.943117178612059),
 (4, 724, 4.118316268486917),
 (5, 889, 5.056882821387941),
 (6, 803, 4.56769055745165),
 (7, 823, 4.681456200227531),
 (8, 862, 4.9032992036405005),
 (9, 927, 5.273037542662116),
 (10, 996, 5.665529010238908),
 (11, 880, 5.005688282138794),
 (12, 676, 3.845278725824801),
 (13, 912, 5.187713310580205),
 (14, 884, 5.02844141069397),
 (15, 718, 4.084186575654153),
 (16, 866, 4.926052332195677),
 (17, 1069, 6.080773606370876),
 (18, 833, 4.738339021615472),
 (19, 727, 4.135381114903299),
 (20, 611, 3.4755403868031856),
 (21, 639, 3.63481228668942),
 (22, 351, 1.9965870307167235)]

Máxima cantidad respecto de semanas

In [84]:
porcentaje_tupla(fraude.map(lambda x: (semanas(x.timestamp), 1)).reduceByKey(lambda a,b: a + b).reduce(lambda a,b: a if a[1] > b[1] else b), cantidad_total)

(17, 1069, 6.080773606370876)

Cantidad respecto de meses

In [85]:
porcentaje_lista(sorted(fraude.map(lambda x: (meses(x.timestamp), 1)).reduceByKey(lambda a,b: a + b).collect()), cantidad_total)

[(1, 3114, 17.713310580204777),
 (2, 3377, 19.209328782707622),
 (3, 3479, 19.78953356086462),
 (4, 3380, 19.226393629124004),
 (5, 3240, 18.43003412969283),
 (6, 990, 5.631399317406143)]

In [86]:
porcentaje_tupla(fraude.map(lambda x: (meses(x.timestamp), 1)).reduceByKey(lambda a,b: a + b).reduce(lambda a,b: a if a[1] > b[1] else b), cantidad_total)

(3, 3479, 19.78953356086462)

Monto respecto del tiempo

Máximo monto respecto de segundos

In [88]:
monto_total = fraude.map(lambda x: (1, x.ammount)).reduceByKey(lambda a,b: a + b).collect()[0][1]
(fraude.map(lambda x: segundos(x.timestamp)).reduce(lambda a,b: a if a > b else b), monto_total, 100)

(13064705, 2614236.338999996, 100)

Máximo monto respecto de minutos

In [68]:
porcentaje_tupla(fraude.map(lambda x: (minutos(x.timestamp), x.ammount)).reduceByKey(lambda a,b: a + b).reduce(lambda a,b: a if a[1] > b[1] else b), monto_total)

(176918, 6239.603999999999, 0.23867788489187583)

Máximo monto respecto de horas

In [69]:
porcentaje_tupla(fraude.map(lambda x: (horas(x.timestamp), x.ammount)).reduceByKey(lambda a,b: a + b).reduce(lambda a,b: a if a[1] > b[1] else b), monto_total)

(3000, 10744.97, 0.411017544194577)

Monto respecto de días

In [70]:
porcentaje_lista(sorted(fraude.map(lambda x: (dias(x.timestamp), x.ammount)).reduceByKey(lambda a,b: a + b).collect()), monto_total)

[(1, 20337.144, 0.777938233686225),
 (2, 15359.606999999995, 0.5875370474681485),
 (3, 14254.337000000001, 0.5452581615269186),
 (4, 14120.447, 0.5401365893873767),
 (5, 16692.721000000005, 0.6385314422790613),
 (6, 12869.08, 0.4922691880613484),
 (7, 21670.363000000027, 0.8289366449664389),
 (8, 13508.152, 0.5167150268122724),
 (9, 13106.597999999994, 0.5013547476359219),
 (10, 12709.721000000001, 0.48617337347784534),
 (11, 21725.538999999993, 0.8310472422057487),
 (12, 14710.558, 0.5627095676294944),
 (13, 10975.38, 0.41983120792354717),
 (14, 15782.409000000001, 0.603710107022578),
 (15, 16438.68099999999, 0.6288138816970218),
 (16, 7279.535, 0.27845741761758946),
 (17, 14338.320000000002, 0.5484706866818603),
 (18, 19745.87399999998, 0.7553209212734462),
 (19, 13647.860999999997, 0.5220591878552422),
 (20, 20877.602, 0.7986118809742409),
 (21, 11047.986000000004, 0.4226085390667513),
 (22, 18677.98300000001, 0.7144718601511277),
 (23, 15191.023999999996, 0.5810883956196135),
 (24,

Máximo monto respecto de días

In [71]:
porcentaje_tupla(fraude.map(lambda x: (dias(x.timestamp), x.ammount)).reduceByKey(lambda a,b: a + b).reduce(lambda a,b: a if a[1] > b[1] else b), monto_total)

(120, 39055.20000000002, 1.4939429697828892)

Monto respecto de semanas

In [72]:
porcentaje_lista(sorted(fraude.map(lambda x: (semanas(x.timestamp), x.ammount)).reduceByKey(lambda a,b: a + b).collect()), monto_total)

[(1, 115303.69900000001, 4.410607307375517),
 (2, 102518.35700000002, 3.9215412727074086),
 (3, 103375.85900000017, 3.9543425151661595),
 (4, 101438.13200000007, 3.8802204103245854),
 (5, 121564.43699999985, 4.6500936119073675),
 (6, 122015.66099999995, 4.6673538723233285),
 (7, 104885.80800000005, 4.012101218060538),
 (8, 118201.74399999983, 4.521463581414933),
 (9, 127460.33400000015, 4.875623986190805),
 (10, 134119.4329999999, 5.130348430980177),
 (11, 108111.65399999979, 4.135496564987499),
 (12, 92534.23399999994, 3.539627715350188),
 (13, 147655.60900000005, 5.64813543432273),
 (14, 146352.54399999988, 5.598290476521454),
 (15, 123198.73099999996, 4.712608770755831),
 (16, 167064.3829999998, 6.390561576536943),
 (17, 158979.2529999999, 6.081288467622366),
 (18, 157838.82199999993, 6.037664599994689),
 (19, 114772.70699999995, 4.390295754357966),
 (20, 105325.59899999997, 4.028924142347795),
 (21, 94819.16799999979, 3.627031213110221),
 (22, 46700.17100000003, 1.7863790776416144)

Máximo monto respecto de semanas

In [76]:
porcentaje_tupla(fraude.map(lambda x: (semanas(x.timestamp), x.ammount)).reduceByKey(lambda a,b: a + b).reduce(lambda a,b: a if a[1] > b[1] else b), monto_total)

(16, 167064.3829999998, 6.390561576536943)

Monto respecto de meses

In [74]:
porcentaje_lista(sorted(fraude.map(lambda x: (meses(x.timestamp), x.ammount)).reduceByKey(lambda a,b: a + b).collect()), monto_total)

[(1, 422636.0470000013, 16.166711505573712),
 (2, 466667.6500000037, 17.85101228370632),
 (3, 462225.6550000017, 17.681096697508742),
 (4, 584271.2670000009, 22.349596258137005),
 (5, 536916.381000001, 20.538172964322865),
 (6, 141519.3389999998, 5.413410290751835)]

Máximo monto respecto de meses

In [77]:
porcentaje_tupla(fraude.map(lambda x: (meses(x.timestamp), x.ammount)).reduceByKey(lambda a,b: a + b).reduce(lambda a,b: a if a[1] > b[1] else b), monto_total)

(4, 584271.2670000009, 22.349596258137005)

Plataformas fraudulentas

In [91]:
identity_rdd = sqlContext.read.csv('identity_features_train.csv', header=True, inferSchema=True).rdd

In [157]:
def unicidad_sistema(string) -> str:
    if string == None:
      return "OTRO"
    lista = ["WINDOWS", "MOTO", "SAMSUNG", "LG", "SM", "GT", "RV", "ZTE", "HUAMWI", "IOS DEVICE","MACOS", "NOKIA", "ALCATEL", "IPHONE" ,"BUILD"]
    for sistema in lista:
        match string.upper().find(sistema):
            case -1:
                continue
            case _:
                return sistema
    return "OTRO"

def unicidad_modelo(string) -> str:
    if string == None:
      return "OTRO"
    lista = ["CHROME", "SAFARI", "FIREFOX", "SAMSUNG", "OPERA", "EDGE", "ANDROID"]
    for modelo in lista:
        match string.upper().find(modelo):
            case -1:
                continue
            case _:
                return modelo
    return "OTRO"

def nulo(numero):
  if numero == None:
    return 0
  else:
    return numero

In [92]:
identity_rdd.first()

Row(_c0=0, Id=2987004, feature01=0.0, feature02=70787.0, feature03=None, feature04=None, feature05=None, feature06=None, feature07=None, feature08=None, feature09=None, feature10=None, feature11=100.0, feature12='NotFound', feature13=None, feature14=-480.0, feature15='New', feature16='NotFound', feature17=166.0, feature18=None, feature19=542.0, feature20=144.0, feature21=None, feature22=None, feature23=None, feature24=None, feature25=None, feature26=None, feature27=None, feature28='New', feature29='NotFound', feature30='Android 7.0', feature31='samsung browser 6.2', feature32=32.0, feature33='2220x1080', feature34='match_status:2', feature35='T', feature36='F', feature37='T', feature38='T', Device='mobile', DeviceMetadata='SAMSUNG SM-G892A Build/NRD90M')

In [145]:
plataformas = identity_rdd.map(lambda x: (x.Id, (x.Device, x.DeviceMetadata, x.feature31))).leftOuterJoin(transactions_rdd.map(lambda x: (x.Id, x.target)))
plataformas_fraudulentas = plataformas.filter(lambda x: x[1][1] == 1).cache()
plataformas_legales = plataformas.filter(lambda x: x[1][1] == 0).cache()

In [144]:
plataformas.first()

(2987040, (('desktop', 'Windows', 'chrome 62.0'), 0))

Fraude por dispositivo

In [172]:
dispositivos = plataformas_fraudulentas.map(lambda x: (x[1][0][0], 1)).reduceByKey(lambda a,b: a+b).leftOuterJoin(plataformas_legales\
.map(lambda x: (x[1][0][0], 1)).reduceByKey(lambda a,b: a+b)).map(lambda x: (x[0], x[1][0], x[1][0] * 100 / (x[1][0] + x[1][1])))
dispositivos.collect()

[(None, 71, 2.417432754511406),
 ('mobile', 4726, 9.861447291544946),
 ('desktop', 4758, 6.369136860141358)]

Máximo dispositivo respecto de la cantidad

In [173]:
dispositivos.reduce(lambda a,b: a if a[1] > b[1] else b)

('desktop', 4758, 6.369136860141358)

Máximo dispositivo respecto del promedio

In [174]:
dispositivos.reduce(lambda a,b: a if a[2] > b[2] else b)

('mobile', 4726, 9.861447291544946)

Fraude por sistema

In [165]:
sistema = plataformas_fraudulentas.map(lambda x: (unicidad_sistema(x[1][0][1]), 1)).reduceByKey(lambda a,b: a+b)\
.leftOuterJoin(plataformas_legales.map(lambda x: (unicidad_sistema(x[1][0][1]), 1)).reduceByKey(lambda a,b: a+b))\
.map(lambda x: (x[0], x[1][0], x[1][0] * 100 / (x[1][0] + nulo(x[1][1]))))
sistema.collect()

[('BUILD', 1035, 16.512444160816848),
 ('MOTO', 377, 15.721434528773978),
 ('MACOS', 218, 1.9339957416607523),
 ('RV', 297, 7.4811083123425695),
 ('GT', 20, 15.037593984962406),
 ('OTRO', 2484, 8.279172082791721),
 ('WINDOWS', 2610, 6.309834638816362),
 ('ZTE', 9, 20.0),
 ('IOS DEVICE', 1079, 6.113314447592068),
 ('SM', 984, 12.479391249207355),
 ('NOKIA', 10, 100.0),
 ('SAMSUNG', 182, 7.882200086617583),
 ('ALCATEL', 7, 13.20754716981132),
 ('LG', 242, 10.980036297640654),
 ('IPHONE', 1, 100.0)]

Máximo sistema respecto de la cantidad

In [166]:
sistema.reduce(lambda a,b: a if a[1] > b[1] else b)

('WINDOWS', 2610, 6.309834638816362)

Máximo sistema respecto del promedio

In [167]:
sistema.reduce(lambda a,b: a if a[2] > b[2] else b)

('IPHONE', 1, 100.0)

Fraude por modelo

In [169]:
modelo = plataformas_fraudulentas.map(lambda x: (unicidad_modelo(x[1][0][2]), 1)).reduceByKey(lambda a,b: a+b)\
.leftOuterJoin(plataformas_legales.map(lambda x: (unicidad_modelo(x[1][0][2]), 1)).reduceByKey(lambda a,b: a+b))\
.map(lambda x: (x[0], x[1][0], x[1][0] * 100 / (x[1][0] + nulo(x[1][1]))))
modelo.collect()

[('ANDROID', 92, 20.444444444444443),
 ('CHROME', 6158, 9.296637932335935),
 ('OTRO', 319, 2.5892857142857144),
 ('SAFARI', 2052, 6.18016444297202),
 ('EDGE', 177, 3.68673193084774),
 ('FIREFOX', 470, 7.4603174603174605),
 ('SAMSUNG', 158, 8.24634655532359),
 ('OPERA', 129, 38.392857142857146)]

Máximo modelo respecto de la cantidad

In [170]:
modelo.reduce(lambda a,b: a if a[1] > b[1] else b)

('CHROME', 6158, 9.296637932335935)

Máximo modelo respecto del promedio

In [171]:
modelo.reduce(lambda a,b: a if a[2] > b[2] else b)

('OPERA', 129, 38.392857142857146)